In [45]:
import datetime as dt
import meteomatics.api as api
from __future__ import print_function
import pandas as pd
import numpy as np

In [46]:
username = 'cantuantunez_juanmanuel'
password = '6Hce5Di66K'

In [47]:
now = dt.datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
startdate_ts = '2024-09-01T00:00:00+00:00'
startdate_ts = dt.datetime.strptime(startdate_ts, '%Y-%m-%dT%H:%M:%S%z')
enddate_ts = startdate_ts + dt.timedelta(days=30)
interval_ts = dt.timedelta(hours=1)

In [48]:
coordinates_ts = [(25.25863754474608, -99.68200619248428)]

In [49]:
parameters_ts = ['grass_land_temperature_sum:C', 'evapotranspiration_24h:mm', 'leaf_wetness:idx', 'soil_moisture_index_-5cm:idx', 'volumetric_soil_water_-5cm:m3m3', 'soil_moisture_index_-15cm:idx','volumetric_soil_water_-15cm:m3m3', 'soil_moisture_index_-50cm:idx', 'volumetric_soil_water_-50cm:m3m3']

In [50]:
print("DataFrame Agricultural Parameters:")
try:
    df = api.query_time_series(coordinates_ts, startdate_ts, enddate_ts, interval_ts,
                                  parameters_ts, username, password)
    print (df.head())
except Exception as e:
    print("Failed, the exception is {}".format(e))

DataFrame Agricultural Parameters:
                                                grass_land_temperature_sum:C  \
lat       lon        validdate                                                 
25.258638 -99.682006 2024-09-01 00:00:00+00:00                        5677.5   
                     2024-09-01 01:00:00+00:00                        5677.5   
                     2024-09-01 02:00:00+00:00                        5677.5   
                     2024-09-01 03:00:00+00:00                        5677.5   
                     2024-09-01 04:00:00+00:00                        5677.5   

                                                evapotranspiration_24h:mm  \
lat       lon        validdate                                              
25.258638 -99.682006 2024-09-01 00:00:00+00:00                       2.74   
                     2024-09-01 01:00:00+00:00                       2.73   
                     2024-09-01 02:00:00+00:00                       2.73   
                   

In [51]:
df.shape

(721, 9)

In [52]:
df.head()

grass_land_temperature_sum:C  \
lat       lon        validdate                                                 
25.258638 -99.682006 2024-09-01 00:00:00+00:00                        5677.5   
                     2024-09-01 01:00:00+00:00                        5677.5   
                     2024-09-01 02:00:00+00:00                        5677.5   
                     2024-09-01 03:00:00+00:00                        5677.5   
                     2024-09-01 04:00:00+00:00                        5677.5   

                                                evapotranspiration_24h:mm  \
lat       lon        validdate                                              
25.258638 -99.682006 2024-09-01 00:00:00+00:00                       2.74   
                     2024-09-01 01:00:00+00:00                       2.73   
                     2024-09-01 02:00:00+00:00                       2.73   
                     2024-09-01 03:00:00+00:00                       2.73   
                     2024-09-01 04:00:00+00:00                       2.73   

                                                leaf_wetness:idx  \
lat       lon        validdate                                     
25.258638 -99.682006 2024-09-01 00:00:00+00:00               1.0   
                     2024-09-01 01:00:00+00:00               1.0   
                     2024-09-01 02:00:00+00:00               1.0   
                     2024-09-01 03:00:00+00:00               1.0   
                     2024-09-01 04:00:00+00:00               1.0   

                                                soil_moisture_index_-5cm:idx  \
lat       lon        validdate                                                 
25.258638 -99.682006 2024-09-01 00:00:00+00:00                         1.293   
                     2024-09-01 01:00:00+00:00                         1.132   
                     2024-09-01 02:00:00+00:00                         1.109   
                     2024-09-01 03:00:00+00:00                         1.090   
                     2024-09-01 04:00:00+00:00                         1.070   

                                                volumetric_soil_water_-5cm:m3m3  \
lat       lon        validdate                                                    
25.258638 -99.682006 2024-09-01 00:00:00+00:00                             0.40   
                     2024-09-01 01:00:00+00:00                             0.38   
                     2024-09-01 02:00:00+00:00                             0.37   
                     2024-09-01 03:00:00+00:00                             0.37   
                     2024-09-01 04:00:00+00:00                             0.37   

                                                soil_moisture_index_-15cm:idx  \
lat       lon        validdate                                                  
25.258638 -99.682006 2024-09-01 00:00:00+00:00                          0.633   
                     2024-09-01 01:00:00+00:00                          0.291   
                     2024-09-01 02:00:00+00:00                          0.299   
                     2024-09-01 03:00:00+00:00                          0.306   
                     2024-09-01 04:00:00+00:00                          0.312   

                                                volumetric_soil_water_-15cm:m3m3  \
lat       lon        validdate                                                     
25.258638 -99.682006 2024-09-01 00:00:00+00:00                              0.26   
                     2024-09-01 01:00:00+00:00                              0.22   
                     2024-09-01 02:00:00+00:00                              0.22   
                     2024-09-01 03:00:00+00:00                              0.23   
                     2024-09-01 04:00:00+00:00                              0.23   

                                                soil_moisture_index_-50cm:idx  \
lat       lon        validdate                                                  
25.25863

In [53]:
df = df.reset_index()
df.columns

Index(['lat', 'lon', 'validdate', 'grass_land_temperature_sum:C',
       'evapotranspiration_24h:mm', 'leaf_wetness:idx',
       'soil_moisture_index_-5cm:idx', 'volumetric_soil_water_-5cm:m3m3',
       'soil_moisture_index_-15cm:idx', 'volumetric_soil_water_-15cm:m3m3',
       'soil_moisture_index_-50cm:idx', 'volumetric_soil_water_-50cm:m3m3'],
      dtype='object')

In [54]:
df2 = df[df['validdate'] == dt.datetime.strptime('2024-09-12T00:00:00+00:00', '%Y-%m-%dT%H:%M:%S%z')]
df2.head()

,lat,lon,validdate,grass_land_temperature_sum:C,evapotranspiration_24h:mm,leaf_wetness:idx,soil_moisture_index_-5cm:idx,volumetric_soil_water_-5cm:m3m3,soil_moisture_index_-15cm:idx,volumetric_soil_water_-15cm:m3m3,soil_moisture_index_-50cm:idx,volumetric_soil_water_-50cm:m3m3
264,25.258638,-99.682006,2024-09-12 00:00:00+00:00,5955.2,4.07,0.0,0.463,0.24,0.702,0.29,0.434,0.24


In [55]:
df2.shape

(1, 12)